In [22]:
import pyrosetta
import pyrosetta_help as ph

logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                ex1=None,
                                                ex2=None,
                                                #mute='all',
                                                ignore_unrecognized_res=True,
                                                load_PDB_components=False,
                                                ignore_waters=False)
                               )

In [2]:
filename = '2x5p.pdb'  # Streptococcus pyogenes fibronectin binding protein Fbab-B = spyCatcher parent
ref:pyrosetta.Pose = pyrosetta.pose_from_file(filename)  # noqa
pyrosetta.rosetta.core.pose.remove_nonprotein_residues(ref)
print( ref.sequence() )
pose = ref.clone()

GAMVDTLDSATHIKFSKRDIDGKELAGATMELRDSSGKTISTWISDGQVKDFYLMPGKYTFVETAAPDGYEVATAITFTVNEQGQVTVNGKATKGDAHIVMVDA


In [23]:
filename = '1ubq.pdb'
pose:pyrosetta.Pose = pyrosetta.pose_from_file(filename)  # noqa
pyrosetta.rosetta.core.pose.remove_nonprotein_residues(pose)
print( pose.sequence() )

MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG


In [6]:
import itertools
prc = pyrosetta.rosetta.core
from isopeptide_scan import IsopetideScanner, IsopetideType

selector = prc.select.residue_selector.LayerSelector()
selector.set_layers(pick_core=True, pick_boundary=True, pick_surface=False)
idxs = prc.select.residue_selector.ResidueVector(selector.apply(pose))

scanner = IsopetideScanner(pose)
df = scanner(idxs)
df.loc[df.verdict == 'favorable']

,Asx,Lys,verdict,score_of_ideal
61,5,13,favorable,-5.736559
616,54,23,favorable,-5.663807
117,15,3,favorable,-5.286383
196,21,56,favorable,-5.227310
598,50,43,favorable,-5.009782
224,23,54,favorable,-4.956847
418,34,30,favorable,-4.741839
33,3,15,favorable,-4.613184
421,34,36,favorable,-4.592618
362,30,34,favorable,-4.180917


In [10]:
df.verdict.value_counts()

verdict
unsuitable     590
unfavorable    125
favorable       97
identity        29
Name: count, dtype: int64

In [31]:

prc = pyrosetta.rosetta.core
pr_res = prc.select.residue_selector

from isopeptide_scan import IsopetideScanner, IsopetideType
from isopeptide_scan.pose_operations import constrain_isopeptide

asx_i = 5
lys_i = 13

def relax_weighted_cart(pose, scorefxn, cycles=5, chi=True, bb=True, jump=True):
    movemap = pyrosetta.MoveMap()
    movemap.set_bb(bb)
    movemap.set_chi(chi)
    movemap.set_jump(jump)
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)  # noqa
    # relax.set_movemap_disables_packing_of_fixed_chi_positions(True)
    relax.set_movemap(movemap)
    relax.minimize_bond_angles(True)
    relax.minimize_bond_lengths(True)
    relax.cartesian(True)
    relax.apply(pose)

MutateResidue = pyrosetta.rosetta.protocols.simple_moves.MutateResidue

class Variant:

    def __init__(self, pose, asx_i: int, lys_i: int):
        self.pose = pose
        self.asx = asx
        self.lys = lys

    @c

def make_isopeptide_mutant(self, asx_i, lys_i):
    mutant = pose.clone()
    
    MutateResidue(target=asx_i, new_res='ASP:SidechainConjugation').apply(mutant)
    MutateResidue(target=lys_i, new_res='LYS:SidechainConjugation').apply(mutant)
    mutant.conformation().declare_chemical_bond(seqpos1=lys_i, atom_name1='NZ', seqpos2=asx_i, atom_name2='CG')
    constrain_isopeptide(mutant, asx_i, lys_i)
    
    iso_selector = pr_res.ResidueIndexSelector()
    iso_selector.append_index(asx_i)
    iso_selector.append_index(lys_i)
    neigh_sele = pr_res.NeighborhoodResidueSelector(selector=iso_selector, distance=6, include_focus_in_subset=False)
    canonical_sele = pr_res.ResiduePropertySelector(prc.chemical.ResidueProperty.CANONICAL_AA)
    
    MutateResidue = pyrosetta.rosetta.protocols.simple_moves.MutateResidue
        
    for idx1 in pr_res.ResidueVector(pr_res.AndResidueSelector(canonical_sele, neigh_sele).apply(pose)):
        if pose.residue(idx1).name3() in ('PRO', 'GLY'):
            continue  # keep as are
        MutateResidue(target=idx1, new_res='ALA').apply(mutant)
    pr_res.NeighborhoodResidueSelector(selector=iso_selector, distance=6, include_focus_in_subset=False)
    relax_weighted_cart(mutant, self.scorefxn_cart_ex, chi=iso_selector.apply(mutant), bb=False, jump=False)
    return mutant

In [32]:
scanner = IsopetideScanner(pose)

mutant = make_isopeptide_mutant(scanner, asx_i = 5, lys_i = 13)

In [37]:
...

Ellipsis

In [9]:
prc = pyrosetta.rosetta.core
pr_res = prc.select.residue_selector

iso_selector = pr_res.ResidueIndexSelector()
iso_selector.append_index(asx_i)
iso_selector.append_index(lys_i)
neigh_sele = pr_res.NeighborhoodResidueSelector(selector=iso_selector, distance=6, include_focus_in_subset=False)
canonical_sele = pr_res.ResiduePropertySelector(prc.chemical.ResidueProperty.CANONICAL_AA)

MutateResidue = pyrosetta.rosetta.protocols.simple_moves.MutateResidue
    
for idx1 in pr_res.ResidueVector(pr_res.AndResidueSelector(canonical_sele, neigh_sele).apply(pose)):
    if pose.residue(idx).name3() in ('PRO', 'GLY'):
        continue  # keep as are
    MutateResidue(target=idx, new_res='ALA').apply(mutant)

NameError: name 'asx_i' is not defined

In [20]:
s.

vector1_unsigned_long[4, 6, 12, 14, 34, 67, 69]

In [19]:
prc.select.residue_selector.NeighborhoodResidueSelector(Exception)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
    2. pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(arg0: pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector)
    3. pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(arg0: pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector)
    4. pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(arg0: pyrosetta.rosetta.utility.vector1_bool, arg1: float)
    5. pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(focus: pyrosetta.rosetta.utility.vector1_bool, distance: float, include_focus_in_subset: bool)
    6. pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(arg0: pyrosetta.rosetta.core.select.residue_selector.ResidueSelector, arg1: float)
  

In [8]:
import json
with open('test-1ubq.json', 'w') as fh:
    json.dump(scores, fh)

In [15]:
import pandas as pd

pd.DataFrame(scores).sort_values('score').head(10)

,A,B,issue,score
5135,68,44,favorable,-8.557301
3391,45,48,favorable,-7.075183
5285,70,42,favorable,-6.037435
1620,22,25,favorable,-5.827465
316,5,13,favorable,-5.736559
4050,54,23,favorable,-5.663807
1066,15,3,favorable,-5.286383
1575,21,56,favorable,-5.227310
138,2,63,favorable,-5.143761
4386,58,55,favorable,-5.047349


vector1_unsigned_long[1, 3, 5, 13, 15, 17, 21, 23, 25, 26, 27, 29, 30, 33, 34, 36, 38, 41, 43, 45, 50, 54, 56, 58, 59, 61, 65, 67, 69]

In [19]:
selector = prc.select.residue_selector.ResidueIndexSelector(43)
prc.simple_metrics.metrics.SasaMetric(selector,prc.scoring.sasa.SasaMethodHPMode.ALL_SASA
                                                               ).calculate(pose)

0.0

In [4]:
# from rdkit import Chem
# block = ph.get_pdbstr(ref)
# protein = Chem.MolFromPDBBlock(block)

In [3]:
from isopeptide_scan import IsopetideScanner, IsopetideType

In [10]:
scanner = IsopetideScanner(pyrosetta.Pose())
ideal: pyrosetta.Pose = scanner.create_ideal_isopeptide(7.4)
print(scanner.scorefxn_cart_ex(ideal))
score = scanner.score_isopeptide_plausibility(ideal.residue(1), ideal.residue(2))
score

-4.530931473077589


-4.533316012880333

In [7]:
scanner = IsopetideScanner(ref)

In [8]:
#ph.constraints2pandas(ideal)

,constraint_name,function_name,funpart,score,atom_1,atom_2,atom_3
0,AtomPairConstraint,HarmonicFunc,7.40/0.20,0.000105,:,NaN,NaN
1,AtomPairConstraint,HarmonicFunc,1.30/0.30,0.009280,:,NaN,NaN
2,AtomPairConstraint,HarmonicFunc,2.40/0.30,0.002254,:,NaN,NaN
3,AtomPairConstraint,HarmonicFunc,3.20/0.30,0.010339,:,NaN,NaN
4,AngleConstraint,CircularHarmonicFunc,2.08/0.20,0.003307,:,A,NaN
5,AngleConstraint,CircularHarmonicFunc,2.08/0.20,0.026461,:,A,NaN
6,DihedralConstraint,CircularHarmonicFunc,3.14/0.20,0.000007,:,A,.
7,DihedralConstraint,CircularHarmonicFunc,3.14/0.20,0.000019,:,A,.


In [9]:
import numpy as np
np.alltrue = np.all
ph.pose2pandas(ideal, scanner.scorefxn_cart).T

,0,1
fa_atr,-1.351528,-1.351528
fa_rep,0.851069,0.851069
fa_sol,1.559859,1.559859
fa_intra_rep,1.086644,3.186784
fa_intra_sol_xover4,0.237275,0.145789
lk_ball_wtd,0.071025,0.071025
fa_elec,-0.752132,-0.752132
hbond_sr_bb,0.0,0.0
hbond_lr_bb,0.0,0.0
hbond_bb_sc,0.0,0.0


In [34]:
import numpy as np
np.alltrue = np.all

print(ph.pose2pandas(ref, scanner.scorefxn_cart).set_index('residue').T[['D117 A ', 'K31 A ']].round(1).to_markdown())

|                       |   D117 A  |   K31 A  |
|:----------------------|----------:|---------:|
| fa_atr                |      -6.4 |     -9.1 |
| fa_rep                |     110.9 |     84.5 |
| fa_sol                |       8.3 |      9.3 |
| fa_intra_rep          |       1.4 |      3.7 |
| fa_intra_sol_xover4   |       0.6 |      0.6 |
| lk_ball_wtd           |       0.4 |     -0.3 |
| fa_elec               |      -0.7 |     -2.6 |
| hbond_sr_bb           |       0   |      0   |
| hbond_lr_bb           |      -0.5 |     -0.6 |
| hbond_bb_sc           |       0   |      0   |
| hbond_sc              |       0   |      0   |
| dslf_fa13             |       0   |      0   |
| atom_pair_constraint  |       0   |      0   |
| coordinate_constraint |       0   |      0   |
| angle_constraint      |       0   |      0   |
| dihedral_constraint   |       0   |      0   |
| omega                 |       0.3 |      0.1 |
| fa_dun                |       4.7 |     14.6 |
| p_aa_pp           

In [13]:
filename = '2x5p.pdb'  # Streptococcus pyogenes fibronectin binding protein Fbab-B = spyCatcher parent
ref:pyrosetta.Pose = pyrosetta.pose_from_file(filename)  # noqa
pyrosetta.rosetta.core.pose.remove_nonprotein_residues(ref)
print( ref.sequence() )
asp117 = ref.pdb_rsd(('A', 117))
lys31 = ref.pdb_rsd(('A', 31))
assert asp117.name3() == 'ASP'
assert lys31.name3() == 'LYS'
scanner = IsopetideScanner(ref)
print('isopeptide residues are correct... ', scanner.is_suitable(lys31, asp117))
d: float = lys31.xyz('CA').distance( asp117.xyz('CA') )
print('residue are at', d, 'Å')
print('score', scanner.score_isopeptide_plausibility(lys31, asp117))

isopeptide residues are correct...  True
residue are at 7.159022419297209 Å
score -1.0980031075230057


In [52]:
ideal.constraint_set().get_all_constraints()[8].get_func().x0()

In [ ]:
pose_ops.relax_weighted_cart(pose, self.scorefxn_cart)

In [26]:
#ph.constraints2pandas(ideal)

cs = ideal.constraint_set()
rows = []
for con in cs.get_all_constraints():
    rows.append(ph.common_ops.constraints.get_constraint_score_data(ideal, con))
pd.DataFrame(rows)

AttributeError: 'pyrosetta.rosetta.core.scoring.constraints.Dihedra' object has no attribute 'atom1'

In [48]:
con.natoms()

2

In [ ]:
# ideal score is very inconsistent!

In [ ]:
import py3Dmol
# from fragmenstein.display import py3Dmol_monkey_patch, divergent_colors

# view = py3Dmol.view()
# view.add_template(carbon_color=color, mol=mol, opacity=0.7)

In [58]:
[n for n in dir(prc.scoring.ScoreType) if 'con' in n]

['_pybind11_conduit_v1_',
 'angle_constraint',
 'atom_pair_constraint',
 'backbone_stub_constraint',
 'backbone_stub_linear_constraint',
 'base_pair_constraint',
 'big_bin_constraint',
 'branch_conn',
 'coarse_chainbreak_constraint',
 'constant_constraint',
 'coordinate_constraint',
 'dihedral_constraint',
 'dock_ens_conf',
 'dof_constraint',
 'dunbrack_constraint',
 'interchain_contact',
 'linear_branch_conn',
 'membrane_span_constraint',
 'membrane_span_term_z_constraint',
 'metalbinding_constraint',
 'metalhash_constraint',
 'pocket_constraint',
 'res_type_constraint',
 'res_type_linking_constraint',
 'sap_constraint',
 'site_constraint',
 'ss_contact_worst']

In [ ]:
help(py3Dmol)

In [ ]:
Chem.MolFromMolFile('/Users/user/iso.mol')

In [26]:
from rdkit import Chem
from typing import Tuple
from rdkit.Chem import AllChem

isopeptide = Chem.MolFromSmiles("C([C@@H](C(=O))N)C(=O)NCCCC[C@@H](C(=O))N")
isopeptide: Chem.Mol = Chem.MolFromPDBBlock(ph.get_pdbstr(test))
isopeptide: Chem.Mol = Chem.MolFromMolFile('/Users/user/iso.mol')    
    
bad_amide = Chem.MolFromSmiles('C(-O)-N')
if isopeptide.HasSubstructMatch(bad_amide):
    # PDB bond order is wrong
    c, o, n = isopeptide.GetSubstructMatch(bad_amide)
    isopeptide.GetBondBetweenAtoms(c, o).SetBondType(Chem.BondType.DOUBLE)
    isopeptide.GetAtomWithIdx(o).SetNumExplicitHs(0)
    isopeptide.GetAtomWithIdx(n).SetNumExplicitHs(1)
    isopeptide.GetAtomWithIdx(o).SetFormalCharge(0)
    isopeptide.GetAtomWithIdx(n).SetFormalCharge(0)

AllChem.SanitizeMol(isopeptide)
isopeptide = AllChem.AddHs(isopeptide, addCoords=True)
isopeptide.RemoveAllConformers()
AllChem.EmbedMolecule(isopeptide)
p = AllChem.MMFFGetMoleculeProperties(isopeptide, 'MMFF94')
ff = AllChem.MMFFGetMoleculeForceField(isopeptide, p, ignoreInterfragInteractions=False)
ff.Initialize()
ff.Minimize()
ff.CalcEnergy()

31.991039401562432

In [27]:
ff.Minimize()
ff.CalcEnergy()

31.991030859356172

In [20]:
p = AllChem.MMFFGetMoleculeProperties(isopeptide, 'MMFF94')
ff = AllChem.MMFFGetMoleculeForceField(isopeptide, p, ignoreInterfragInteractions=False)
ff.Initialize()
bb: Tuple[Tuple[int, ...]]
bi: int
for bb in isopeptide.GetSubstructMatches( Chem.MolFromSmiles('NCC(=O)') ):
    for bi in bb:
        ff.AddFixedPoint(bi)
ff.CalcEnergy()

ValueError: Bad Conformer Id

In [18]:
ff.Minimize()
ff.CalcEnergy()

54.62919603372348

In [ ]:
Chem.MolToMolFile(isopeptide, 'test.mol')

In [41]:
asx = asp117

for i in range(1, 1+asx.natoms()):
    print(asx.atom_name(i))

 N  
 CA 
 C  
 O  
 CB 
 CG 
 OD1
 OD2
 H  
 HA 
1HB 
2HB 


isopeptide residues are correct...  True


27.00276003938617

In [ ]:
test = scanner.ideal_template.clone()
scanner.copy_backbone(donor_residue=asp117, acceptor_residue=test.residue(1))
scanner.copy_backbone(donor_residue=lys31, acceptor_residue=test.residue(2))

In [ ]:
from isopeptide_scan import PoseOps

test_ops = PoseOps(test)
test_ops.relax_sidechains(scanner.scorefxn, 3)
test.dump_pdb('test.pdb')

In [ ]:
scanner.scorefxn(test)

In [ ]:
scanner.is_suitable(26, 67)

In [ ]:
ideal: pyrosetta.Pose = scanner.create_ideal_isopeptide(8)
scanner.score_isopeptide(ideal.residue(1), ideal.residue(2))

In [ ]:
trash = pyrosetta.pose_from_sequence('DK')

In [ ]:
[c.to_string().split('\n')[0] for c in ideal.constraint_set().get_all_constraints()]

In [ ]:
c = _[1]

In [ ]:
c

In [ ]:
TestIsopetide§a

In [ ]:
NO LONGER VALID


ori_test = create_test()
print(ori_test.sequence())

ori_test.remove_constraints()
constrain_isopeptide(ori_test)
constrain_CA_distance(ori_test, 9)
scorefxn=pyrosetta.create_score_function('ref2015_cart')
scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.dihedral_constraint, 5)
movemap = pyrosetta.MoveMap()
movemap.set_bb(True)
movemap.set_chi(True)
movemap.set_jump(True)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
#relax.set_movemap_disables_packing_of_fixed_chi_positions(True)
relax.set_movemap(movemap)
relax.apply(ori_test)

ori_test.remove_constraints()
constrain_isopeptide(ori_test)

In [ ]:
# ## Show
import nglview as nv

nv.show_rosetta(test)

In [ ]:

    
    
scorefxn=pyrosetta.create_score_function('ref2015_cart')
scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.dihedral_constraint, 5)
scores = []
for i, j in itertools.product(ph.pose_range(ref, protein_only=True), repeat=2):
    if i == j:
        continue
    s = quick_score_layout(asx=ref.residue(i), lys=ref.residue(j))
    scores.append((i, j, s))

In [ ]:
import pandas as pd

df = pd.DataFrame([dict(zip(['asx', 'lys', 'score'], s)) for s in scores])
# px.imshow expects wide not long hence the df.pivot
fig = px.imshow(df.pivot(index='asx',columns='lys').score,
                color_continuous_scale='teal_r', range_color=[5, 20],
                title='Potentially suitable asx-lys isopeptide resiude pairs',
               )
fig.update_layout(title_x=0.5)
fig

In [ ]:
df.loc[df.score < 10].sort_values('score')

In [ ]:
asx_i = 54
lys_i = 23
mutant = ref.clone()
MutateResidue = pyrosetta.rosetta.protocols.simple_moves.MutateResidue
MutateResidue(target=asx_i, new_res='ASP:SidechainConjugation').apply(mutant)
MutateResidue(target=lys_i, new_res='LYS:SidechainConjugation').apply(mutant)
mutant.conformation().declare_chemical_bond(seqpos1=lys_i, atom_name1='NZ', seqpos2=asx_i, atom_name2='CG')
constrain_isopeptide(mutant, asx_i, lys_i)

movemap = pyrosetta.MoveMap()
movemap.set_bb(False)
movemap.set_chi(True)
movemap.set_jump(True)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
#relax.set_movemap_disables_packing_of_fixed_chi_positions(True)
relax.set_movemap(movemap)
relax.apply(ori_test)


In [ ]:
rts = test.residue_type_set_for_pose()
rts.get_base_types_name3('ASP')[1]

In [ ]:
ref.residue(1)

In [ ]:
# pyrosetta.rosetta.core.util.add_covalent_linkage(pose=test, 
#                                                  resA_pos=1, resB_pos=2, 
#                                                  resA_At=test.residue(1).atom_index('CG'), 
#                                                  resB_At=test.residue(2).atom_index('NZ'), 
#                                                  remove_hydrogens=True)

```
AtomPair NZ 9A CG 121A HARMONIC 1.30 0.3
AtomPair CE 9A CG 121A HARMONIC 2.40 0.3
AtomPair 1HZ 9A OD 121A HARMONIC 3.20 0.3
Angle CE 9A NZ 9A CG 121A CIRCULARHARMONIC 2.08 0.2 #119 deg
Angle NZ 9A CG 121A OD 121A CIRCULARHARMONIC 2.08 0.2
Dihedral CE 9A NZ 9A CG 121A CB 121A CIRCULARHARMONIC 3.14159 0.2
Dihedral 1HZ 9A NZ 9A CG 121A OD 121A CIRCULARHARMONIC 3.14159 0.2
```

cst_fa_weight 5

In [ ]:
#Angle CE 9A NZ 9A CG 121A CIRCULARHARMONIC 2.08 0.2 #119 deg
#Angle NZ 9A CG 121A OD 121A CIRCULARHARMONIC 2.08 0.2

In [ ]:
test_ref = test.clone()

In [ ]:
import numpy as np
cycles = 15
movemap = pyrosetta.MoveMap()
movemap.set_bb(True)
movemap.set_chi(True)
movemap.set_jump(True)
scorefxn.set_weight(pyrosetta.rosetta.core.scoring.ScoreType.atom_pair_constraint, 5)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
#relax.set_movemap_disables_packing_of_fixed_chi_positions(True)
relax.set_movemap(movemap)
test_scores = {}
for i in np.arange(0, 20.5, 0.1):
    test = test_ref.clone()
    test.remove_constraints()
    constrain_CA_distance(test, i)
    constrain_isopeptide(test)
    relax.apply(test)
    test_scores[i] = scorefxn(test)

In [ ]:
# distances

test_scores = {0.0: 829.5172358840177,
         0.1: 777.9509751175779,
         0.2: 732.3929631395587,
         0.30000000000000004: 690.5260730375543,
         0.4: 648.9792405060249,
         0.5: 608.4890636937826,
         0.6000000000000001: 559.6898360629772,
         0.7000000000000001: 786.8215729408977,
         0.8: 492.4208966480116,
         0.9: 460.69782700073995,
         1.0: 416.1464918019651,
         1.1: 388.2167777023903,
         1.2000000000000002: 355.7450217050791,
         1.3: 324.9045948803566,
         1.4000000000000001: 294.0022992850152,
         1.5: 471.3526100169143,
         1.6: 435.39255033487046,
         1.7000000000000002: 400.8312641648571,
         1.8: 293.99621165753734,
         1.9000000000000001: 165.30806028234093,
         2.0: 238.20084971264444,
         2.1: 126.27946820212944,
         2.2: 173.5091105496658,
         2.3000000000000003: 92.27074797734963,
         2.4000000000000004: 152.10642654766116,
         2.5: 90.05100643079703,
         2.6: 116.83613775709578,
         2.7: 101.83154271723377,
         2.8000000000000003: 64.21554588287029,
         2.9000000000000004: 57.432237087028554,
         3.0: 51.313599118934846,
         3.1: 101.23965703424105,
         3.2: 40.975858535302024,
         3.3000000000000003: 42.490549323505235,
         3.4000000000000004: 66.88597038861194,
         3.5: 23.565466644544614,
         3.6: 53.32739774146941,
         3.7: 27.293769761136915,
         3.8000000000000003: 44.014241113292854,
         3.9000000000000004: 44.586358393321866,
         4.0: 39.75042010163031,
         4.1000000000000005: 35.580049409262905,
         4.2: 31.929280933765483,
         4.3: 28.47305583076133,
         4.4: 25.777379192084094,
         4.5: 23.760757899099815,
         4.6000000000000005: 21.992139570105163,
         4.7: 20.447550739101732,
         4.800000000000001: 19.101640456119974,
         4.9: 17.91782606424314,
         5.0: 16.872413827211243,
         5.1000000000000005: 15.628627942201643,
         5.2: 14.991896915243577,
         5.300000000000001: 14.468741353220324,
         5.4: 13.969408840194262,
         5.5: 13.552741222367473,
         5.6000000000000005: 13.181755789562168,
         5.7: 12.760197270219331,
         5.800000000000001: 12.384874316780248,
         5.9: 12.039754944832548,
         6.0: 11.691715175282901,
         6.1000000000000005: 11.389567668692294,
         6.2: 11.137846412576707,
         6.300000000000001: 10.965292916235384,
         6.4: 10.930483471931876,
         6.5: 5.687119188705657,
         6.6000000000000005: 10.727061644112414,
         6.7: 10.66582620779735,
         6.800000000000001: 5.058878547507912,
         6.9: 7.903203781574227,
         7.0: 10.53736860030605,
         7.1000000000000005: 6.986290802337589,
         7.2: 6.5703075833452615,
         7.300000000000001: 6.159069125681946,
         7.4: 5.7583176678849615,
         7.5: 5.38141076959865,
         7.6000000000000005: 5.105205369254645,
         7.7: 4.92105048841845,
         7.800000000000001: 4.815269309315882,
         7.9: 4.785489651553809,
         8.0: 4.8338525331682645,
         8.1: 4.9716743084016635,
         8.200000000000001: 5.20901218358933,
         8.3: 2.816926553493541,
         8.4: 2.8335178535559598,
         8.5: 10.141942731206917,
         8.6: 10.147537198424269,
         8.700000000000001: 10.18202979989414,
         8.8: 4.169984320553877,
         8.9: 10.375371764543688,
         9.0: 2.3436413248157426,
         9.1: 10.803130917037308,
         9.200000000000001: 11.177226525203402,
         9.3: 2.536891665835642,
         9.4: 2.5667919719431986,
         9.5: 2.5936313706976866,
         9.600000000000001: 14.772342283398428,
         9.700000000000001: 16.395965438330794,
         9.8: 17.427473961401837,
         9.9: 1.3579585406107113,
         10.0: 23.748292072202197,
         10.100000000000001: 27.156144494998177,
         10.200000000000001: 31.107307264996265,
         10.3: 13.771965165801838,
         10.4: 18.43083441517254,
         10.5: 25.345635010272446,
         10.600000000000001: 29.406826338911387,
         10.700000000000001: 37.48267780225471,
         10.8: 45.44704475757986,
         10.9: 55.2294427602269,
         11.0: 64.22131918421444,
         11.100000000000001: 79.69099353592917,
         11.200000000000001: 61.62254565415627,
         11.3: 97.94545641149656,
         11.4: 110.35589146746877,
         11.5: 124.33157176221114,
         11.600000000000001: 137.63351619767053,
         11.700000000000001: 162.31112014557144,
         11.8: 175.8288148928701,
         11.9: 192.52820534677232,
         12.0: 171.456957826006,
         12.100000000000001: 225.43287944537593,
         12.200000000000001: 250.22293233992042,
         12.3: 227.35395892968015,
         12.4: 247.80749110719194,
         12.5: 321.77716008700577,
         12.600000000000001: 291.3686828836881,
         12.700000000000001: 369.88324344781694,
         12.8: 395.04391040963765,
         12.9: 363.7130142267156,
         13.0: 448.21519881477275,
         13.100000000000001: 416.48982126762155,
         13.200000000000001: 505.8576550865257,
         13.3: 472.8726876708307,
         13.4: 502.3272910416922,
         13.5: 599.4826834820997,
         13.600000000000001: 632.4953763476223,
         13.700000000000001: 665.4329043342739,
         13.8: 700.7783588876678,
         13.9: 733.9725082518262,
         14.0: 772.7276385038747,
         14.100000000000001: 810.0604634187073,
         14.200000000000001: 850.4640468026687,
         14.3: 888.9689669490729,
         14.4: 928.1824058258952,
         14.5: 968.4445297575563,
         14.600000000000001: 1012.5419714233874,
         14.700000000000001: 1053.759727256388,
         14.8: 1098.8321794667354,
         14.9: 1145.126471452397,
         15.0: 1188.4505740592087,
         15.100000000000001: 1146.9242085764422,
         15.200000000000001: 1192.4409922173406,
         15.3: 1240.5738930773093,
         15.4: 1380.6901779936206,
         15.5: 1336.7557314193198,
         15.600000000000001: 1387.7044637575113,
         15.700000000000001: 1437.6531678535591,
         15.8: 1489.4793095744662,
         15.9: 1642.2884530350325,
         16.0: 1698.2788817037244,
         16.1: 1650.6665664819147,
         16.2: 1706.9968825237088,
         16.3: 1762.62318356463,
         16.400000000000002: 1821.0056055311734,
         16.5: 1878.9077852361213,
         16.6: 1954.0931098151664,
         16.7: 2011.9999981651506,
         16.8: 2068.834293687462,
         16.900000000000002: 2134.958170487931,
         17.0: 2185.950693540045,
         17.1: 2253.463329684435,
         17.2: 2313.1488557924517,
         17.3: 2390.4065227910555,
         17.400000000000002: 2445.7311213986295,
         17.5: 2514.419767142029,
         17.6: 2593.504216919098,
         17.7: 2661.2944872660973,
         17.8: 2740.9058466777433,
         17.900000000000002: 2794.2466093153107,
         18.0: 2877.2317207278966,
         18.1: 2959.078618179139,
         18.2: 3029.5535740496143,
         18.3: 3089.839997494769,
         18.400000000000002: 3165.8818107270304,
         18.5: 3242.9608211598897,
         18.6: 3340.618151430879,
         18.7: 3400.0156831017493,
         18.8: 3479.8695551475257,
         18.900000000000002: 3575.032755648972,
         19.0: 3642.6622744044384,
         19.1: 3737.7017136546283,
         19.200000000000003: 3808.7287340627227,
         19.3: 3911.7721919834667,
         19.400000000000002: 3996.9171030142875,
         19.5: 4077.408588021188,
         19.6: 4152.5139368385835,
         19.700000000000003: 4240.964698228315,
         19.8: 4350.863340742675,
         19.900000000000002: 4443.677283564195,
         20.0: 4512.505339784237,
         20.1: 4631.230889139941,
         20.200000000000003: 4719.312852377857,
         20.3: 4810.071483180874,
         20.400000000000002: 4904.885431003954}

In [ ]:
import plotly.express as px

fig = px.line(pd.DataFrame({'score': test_scores.values(), 'distance': test_scores.keys()}),
              title='Asx-Lys Cα Distance vs. ∆∆G w/ constraits',
              x='distance', y='score', log_y=True)
fig

In [ ]:
1

In [ ]:
pyrosetta.rosetta.core.chemical.

In [ ]:
pyrosetta.rosetta.core.chemical.VariantType.DEPROTONATED

In [ ]:
pyrosetta.pose_from_sequence('E[Glu:Protonated]')

In [ ]:
import importlib_resources

In [ ]:
# /usr/local/Caskroom/miniconda/base/envs/biochem38/lib/python3.8/site-packages/pyrosetta/database/chemical/residue_type_sets/fa_standard/patches 

In [ ]:
!grep -r 'proton' /usr/local/Caskroom/miniconda/base/envs/biochem38/lib/python3.8/site-packages/pyrosetta/database/chemical/residue_type_sets/fa_standard/patches

In [62]:
ideal.residue(1) #.atom_index('CB')

In [60]:
_.

5